## Set-up

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from pprint import pprint
import re
from tqdm import tqdm
tqdm.pandas()
import numpy as np

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=12, progress_bar=True)

import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

import warnings
warnings.filterwarnings("ignore")

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
#!pwd
import os

# Google Bucket
# file name checkpoint_0512_sent_split.parquet
path_bucket = 'gs://msca-sp23-bucket/nlp_data'
runtime_path = '/home/anthony/projects/nlp_runtime'
bucket_read = path_bucket + '/' + 'checkpoint_0525_full_article_split.parquet'

os.chdir(runtime_path)
print(os.getcwd())

/home/anthony/projects/nlp_runtime


In [3]:
import spacy

2023-05-25 20:55:50.882505: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 20:55:51.793787: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import en_core_web_md
nlp = spacy.load("en_core_web_md")

## Preparation

In [5]:
df_raw = pd.read_parquet(bucket_read, engine='pyarrow')
df_raw.head(3)

,url,date,title,text_cleaned,article_topic,domain,article_id,sentences,entities
0,https://theenterpriseleader.com/2020/01/01/decentralized-machine-learning-reaches-market-cap-of-15919-00-dml.html,2020-01-01,"Decentralized Machine Learning Reaches Market Cap of $15,919.00 (DML) - Enterprise Leader","Decentralized Machine Learning Reaches Market Cap of 15,919.00 DML Enterprise Leader Daily Ratings News for Decentralized Machine Learning Complete the form below to receive the latest headlines and analysts' mendations for Decentralized Machine Learning with our free daily email newsletter: Follow EnterpriseLeade Recent Posts GE Announces Its Plan To Sell The Distributed Power Business To Advent International Ebang Communication Resorts To The Filing Of An IPO In Hong Kong How to Open DAA, ...",3,theenterpriseleader.com,1,"[Decentralized Machine Learning Reaches Market Cap of 15,919.00 DML Enterprise Leader Daily Ratings News for Decentralized Machine Learning Complete the form below to receive the latest headlines and analysts' mendations for Decentralized Machine Learning with our free daily email newsletter: Follow EnterpriseLeade Recent Posts GE Announces Its Plan To Sell The Distributed Power Business To Advent International Ebang Communication Resorts To The Filing Of An IPO In Hong Kong How to Open DAA,...","[[Decentralized Machine Learning Reaches Market Cap, ORG], [Decentralized Machine Learning, ORG], [Hong Kong, GPE], [NRG, ORG], [IMG, ORG], [Mistake Foxconn Launches Investigation After Reports Of Harsh Working Conditions, ORG], [Flagstar Bancorp, ORG], [Wells Fargo UK, ORG], [Royal Bank Of Scotland RBS, ORG], [France, GPE], [China, GPE], [FII IPO Uber, ORG], [Paris Deutsche Bank Plans To Slash, ORG], [Samsung, ORG], [Apple Back In Court, ORG], [Apple, ORG], [Goldman Sachs, ORG], [JPMorgan C..."
1,https://marketreportobserver.com/artificial-intelligence-ai-in-marketing-market-scope-size-share-trends-forecast-by-2026/,2020-01-01,"Artificial Intelligence (AI) in Marketing Market Scope, Size, Share, Trends, Forecast by 2026 – Market Reports Observer","Artificial Intelligence AI in Marketing Market Scope, Size, Share, Trends, Forecast by 2026 Market Reports Observer Skip to content 488 E 11th Ave, Eugene, OR 97401, USA 1 541 687 2347 email protected Market Reports Observer Artificial intelligence AI is the ability to process information in a way similar to the thought process of humans in learning, decision making, and problem solving. Enterprises are now identifying the value associated with integrating AI into their business processes. ...",1,marketreportobserver.com,2,"[ Artificial Intelligence AI in Marketing Market Scope, Size, Share, Trends, Forecast by 2026 Market Reports Observer Skip to content 488 E 11th Ave, Eugene, OR 97401, USA 1 541 687 2347 email protected Market Reports Observer Artificial intelligence AI is the ability to process information in a way similar to the thought process of humans in learning, decision making, and problem solving., Enterprises are now identifying the value associated with integrating AI into their business processes...","[[USA, GPE], [Market Reports Observer Artificial, ORG], [North America, LOC], [South America, LOC], [Europe, LOC], [Asia Pacific, LOC], [Middle East Africa, LOC], [North America, LOC], [North America, LOC], [Asia Pacific, LOC], [Japan, GPE], [China, GPE], [India, GPE], [Europe, LOC], [Middle East Africa, LOC], [South America, LOC], [Artificial Intelligence AI in Marketing Market, ORG], [IBM Corporation, ORG], [Google Inc., ORG], [Intel Corporation, ORG], [Microsoft Corporation, ORG], [NVIDIA..."
2,https://www.themarketsdaily.com/2020/01/01/lisk-machine-learning-24-hour-trading-volume-tops-10-00-lml.html,2020-01-01,Lisk Machine Learning 24 Hour Trading Volume Tops $10.00 (LML) - Markets Daily,"Lisk Machine Learning 24 Hour Trading Volume Tops 10.00 LML Markets Daily Home Privacy Policy About/ Contact Business Stocks Technology Health Lifestyle Politics Daily Ratings

## Quick look at sentences and entities

Structure of columns: sentences, entities

In [6]:
test = df_raw.head(1)
sents = test['sentences'].values[0]
entities = test['entities'].values[0]

In [7]:
# check null / nan value in the two columns
print(f'Number of null values in sentences: {df_raw["sentences"].isnull().sum()}')
print(f'Number of null values in entities: {df_raw["entities"].isnull().sum()}')

Number of null values in sentences: 0
Number of null values in entities: 0


In [8]:
print(f'There are {len(sents)} sentences for article 1')
print(f'There are {len(entities)} entities for article 1')

There are 28 sentences for article 1
There are 60 entities for article 1


## Breakdown by sentences

In [9]:
# select the necessary columns
df_sentences = df_raw[['date','title', 'article_id', 'sentences']]
df_sentences.head(3)

,date,title,article_id,sentences
0,2020-01-01,"Decentralized Machine Learning Reaches Market Cap of $15,919.00 (DML) - Enterprise Leader",1,"[Decentralized Machine Learning Reaches Market Cap of 15,919.00 DML Enterprise Leader Daily Ratings News for Decentralized Machine Learning Complete the form below to receive the latest headlines and analysts' mendations for Decentralized Machine Learning with our free daily email newsletter: Follow EnterpriseLeade Recent Posts GE Announces Its Plan To Sell The Distributed Power Business To Advent International Ebang Communication Resorts To The Filing Of An IPO In Hong Kong How to Open DAA,..."
1,2020-01-01,"Artificial Intelligence (AI) in Marketing Market Scope, Size, Share, Trends, Forecast by 2026 – Market Reports Observer",2,"[ Artificial Intelligence AI in Marketing Market Scope, Size, Share, Trends, Forecast by 2026 Market Reports Observer Skip to content 488 E 11th Ave, Eugene, OR 97401, USA 1 541 687 2347 email protected Market Reports Observer Artificial intelligence AI is the ability to process information in a way similar to the thought process of humans in learning, decision making, and problem solving., Enterprises are now identifying the value associated with integrating AI into their business processes..."
2,2020-01-01,Lisk Machine Learning 24 Hour Trading Volume Tops $10.00 (LML) - Markets Daily,3,"[ Lisk Machine Learning 24 Hour Trading Volume Tops 10.00 LML Markets Daily Home Privacy Policy About/ Contact Business Stocks Technology Health Lifestyle Politics Daily Ratings News for Lisk Machine Learning Complete the form below to receive the latest headlines and analysts' mendations for Lisk Machine Learning with our free daily email newsletter: Follow TheMarketsDaily Recent Posts TERA Price Down 14.5 Over Last 7 Days TERA Mindexcoin MIC Hits 1 Day Trading Volume of 1,347.00 Garlicoin ..."


In [10]:
# explode the dataframe by sentence
df_sentences_exp = df_sentences.explode('sentences')

In [11]:
print(f'Previous number of rows: {len(df_sentences)}')
print(f'New number of rows: {len(df_sentences_exp)}')

Previous number of rows: 133258
New number of rows: 4489276


In [12]:
# sort by date
df_sentences_exp.sort_values(by=['article_id'], inplace=True)

In [13]:
# drop duplicates
df_sentences_exp.drop_duplicates(subset='sentences', inplace=True, keep='first')
print(f'New number of rows: {len(df_sentences_exp)}')

New number of rows: 2223016


In [14]:
df_sentences_exp.sample(5)

,date,title,article_id,sentences
57839,2022-01-18,"The Poisson Multinomial Distribution and Its Applications in Voting Theory, Ecological Inference, and Machine Learning - NewsBreak",57840,"In spite of this, the use of image analysis in the development of these systems is still recent and underexplored, especially in materials often studied via optical imaging techniques such as liquid crystals."
47248,2021-10-22,GoldSpot Discoveries Corp.: GoldSpot Discoveries Engaged by Ranchero Gold to Apply AI on the Santa Daniela Project in Sonora Mexico,47249,"Our leading team of expert scientist's merge geoscience and data science to deliver bespoke solutions that save time, reduce costs and produce far more accurate results than ever before possible."
73972,2022-06-29,Artificial Intelligence Investments Will Likely Push Aterian’s Price Up (ATER) | Seeking Alpha,73973,Investors that purchase shares of mon stock during a short squeeze may lose a significant portion or all of their investment.
21756,2020-12-06,"Global Artificial Neural Network Software Market 2026 The leading Industry Players : GMDH, Artificial Intelligence Techniques, Oracle, IBM, Microsoft etc. – Canaan Mountain Herald",21757,"USA: 1 972 362 8199 IND: 91 895 659 5155 Post navigation Non Contact Temperature Sales Market Report 2020 Analysis On Trends by various Key Plares: Fluke, Accurate Sensors, OMRON, IFM Electronic, Turck Non Volatile Dual In Line Memory Module NVDIMM Sales Market Report 2020 Current Trends, Future Aspect Analysis by Top Competitors: Viking Technology Inc., AgigA Tech Inc.?,"
7776,2020-04-22,"Edge AI Software Market 2020, Global Industry Analysis, Size, Share, Growth, Trends and Forecast - 2025",7777,13Key Players Profiles13.1 IBM13.1.1 IBM Company Details13.1.2 IBM Business Overview and Its Total Revenue13.1.3 IBM Edge AI Software Introduction13.1.4 IBM Revenue in Edge AI Software Business 2015 2020 13.1.5 IBM Recent Development13.2 Microsoft13.2.1 Microsoft Company Details13.2.2 Microsoft Business Overview and Its Total Revenue13.2.3 Microsoft Edge AI Software Introduction13.2.4 Microsoft Revenue in Edge AI Software Business 2015 2020 13.2.5 Microsoft Recent Development13.3 Intel13.3.1...


In [36]:
# Get the description of the sentence lengths
sent_len = df_sentences_exp['sentences'].str.len().describe()

# Convert scientific notation to actual number format
pd.options.display.float_format = '{:.2f}'.format

# Print the updated description
print(sent_len)

count   2223016.00
mean        281.37
std         537.98
min           1.00
25%         101.00
50%         156.00
75%         246.00
max       46098.00
Name: sentences, dtype: float64


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [22]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)


In [32]:
test = df_sentences_exp.sample(1)['sentences'].values[0]
print(test)
print(pipe(test))

Roey is the CTO and a Co founder of BeyondMinds, a start up that helps enterprises achieve sustainable value from AI.
[{'label': 'neutral', 'score': 0.9998063445091248}]


In [34]:
# save a checkpoint
df_sentences_exp.to_parquet(path_bucket + '/' +'checkpoint_0525_full_article_split_sentences.parquet', engine='pyarrow')

## Methodology: How to measure sentiments within an article.

As we have split sentences with SpaCy for each article, and the sentences are exploded into a dataframe. Now what we have is a dataframe of sentences instead of a df of articles. 

The primary idea is to use aggregated sentiment scores with sentences, rather than the overall sentiment of an article. The benefit of such method is that:
1. it reduces token size for input; this is particularly helpful when we are using transformer and Bert-based models
2. it is more accurate by aggregating the sentiment of sentences because 
    - full article sentiment has a lot of noises (e.g. unclean text like other titles/ads text from parsing)
    - full article is computationally expensive for text embedding

# Reference
1. Sentiment Analysis in 10 Minutes with BERT and TensorFlow. https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671